# Lab 4: Functions and Functional Programming (Part 2)
Look at you go! Congratulations on making it to the second part of the lab!

These assignments are *absolutely not required*! Even if you're here, you may find it more valuable to skim the problems here and attempt the problems that are most interesting to you - and that's perfectly fine. Don't feel any need to complete them in sequential order at this point.

I'm honestly WAYYYY more psyched about the functional programming stuff than the functions stuff (don't tell anyone &#128064;) so let's start there!

## Building Decorators

### Automatic Caching
In class, we wrote a decorator `memoize` that will automatically caches any calls to the decorated function. You can assume that all arguments passed to the decorated function will always be hashable types.

```Python
def memoize(function):
    cache = {}
    def memoized_fn(*args):
        if args not in cache:
            cache[args] = function(*args)
        return cache[args]
    return memoized_fn
```

We saw how one use case for this in class:

```Python
@memoize
def fib(n):
    return fib(n-1) + fib(n-2) if n > 2 else 1

fib(10)  # 55 (takes a moment to execute)
fib(10)  # 55 (returns immediately)
fib(100) # doesn't take forever
fib(400) # doesn't raise RuntimeError
```

#### Cache Options (Challenge)

Add `maxsize` and `eviction_policy` keyword arguments, with reasonable defaults (perhaps `maxsize=None` as a sentinel), to your `cache` decorator. `eviction_policy` should be one of `'LRU'`, `'MRU'`, or `'random'`. It can be tricky to figure out how to construct a decorator with arguments.

Also, add function attributes called `.cache_info` and `.cache_clear` which can be called to get aggregate statistics about the cache  and clear the cache, respectively.

*Note*: This caching decorator (with arguments!) is actually implemented as part of the language in `functools.lru_cache`

In [ ]:
def memoize(???):
    pass

@memoize(???)
def fib(n):
    return fib(n-1) + fib(n-2) if n > 2 else 1

### Better Debugging Decorator
The `debug` decorator we wrote in class isn't very good. It doesn't tell us which function is being called, and it just dumps a tuple of positional arguments and a dictionary of keyword arguments - it doesn't even know what the names of the positional arguments are! If the default arguments aren't overridden, it won't show us their value either.

Use function attributes to improve our `debug` decorator into a `print_args` decorator that is "as good as you can make it."

```Python
def print_args(function):
    def wrapper(*args, **kwargs):
        # (1) You could do something here
        retval = function(*args, **kwargs)
        # (2) You could also do something here
        return retval
    return wrapper
```

*Hint: Consider using the attributes `fn.__name__` and `fn.__code__`. You'll have to investigate these attributes, but I will say that the `fn.__code__` code object contains a number of useful attributes - for instance, `fn.__code__.co_varnames`. Check it out! More information on function attributes is available in the latter half of Lab 3.*

#### Note
There are a lot of subtleties to this function, since functions can be called in a number of different ways. How does your `print_args` handle keyword arguments or even keyword-only arguments? Variadic positional arguments? Variadic keyword arguments? For more customization, look at `fn.__defaults__`, `fn.__kwdefaults__`, as well as other attributes of `fn.__code__`.

In [ ]:
def print_args(???):
    pass

@print_args(???)
def is_prime(n):
    for i in range(2, n):
        if n % i == 0:
            return False
    return True

print(is_prime(198239813))
print(is_prime(4028769383))

@print_args(???)
def stylize_quote(quote, **kwargs):
    print('> {}'.format(quote))
    print('-'*(len(quote) + 2))
    
    for k, v in kwargs.items():
        print('{k}: {v}'.format(k=k, v=v))

stylize_quote('Doth mother know you weareth her drapes?', speaker='Iron Man', year='2012', movie='The Avengers')

@print_args(???)
def draw_table(num_rows, num_cols):
    sep = '+' + '+'.join(['-'] * num_cols) + '+'
    line = '|' + '|'.join([' '] * num_cols) + '|'
    
    for _ in range(num_rows):
        print(sep)
        print(line)
    print(sep)
    
draw_table(10, 10)
draw_table(3, 8)

### Dynamic Type Checker (challenge)

Functions in Python can be optionally annotated by semantically-useless but structurally-valuable type annotations. For example:

```Python
def foo(a: int, b: str) -> bool:
    return b[a] == 'X'

foo.__annotations__  # => {'a': int, 'b': str, 'return': bool}
```

Write a runtime type checker, implemented as a decorator, that enforces that the types of arguments and the return value are valid.

```Python
def enforce_types(function):
    pass  # Your implementation here
```

For example:

```Python
@enforce_types
def foo(a: int, b: str) -> bool:
    if a == -1:
        return 'Gotcha!'
    return b[a] == 'X'

foo(3, 'abcXde')  # => True
foo(2, 'python')  # => False
foo(1, 4)  # prints "Invalid argument type for b: expected str, received int
foo(-1, '')  # prints "Invalid return type: expected bool, received str
```

There are lots of nuances to this function. What happens if some annotations are missing? How are keyword arguments and variadic arguments handled? What happens if the expected type of a parameter is not a primitive type? Can you annotate a function to describe that a parameter should be a list of strings? A tuple of (str, bool) pairs? A dictionary mapping strings to lists of integers? Read more about [advanced type hints](https://docs.python.org/3/library/typing.html) from the documentation.

As you make progress, show your decorator to a member of the course staff. 

In [ ]:
def enforce_types(function):
    pass # Your implementation here

@enforce_types
def foo(a: int, b: str) -> bool:
    if a == -1:
        return 'Gotcha!'
    return b[a] == 'X'

foo(3, 'abcXde')  # => True
foo(2, 'python')  # => False
foo(1, 4)  # prints "Invalid argument type for b: expected str, received int
foo(-1, '')  # prints "Invalid return type: expected bool, received str

## Nested Functions and Closures

In class, we saw that a function can be defined within the scope of another function. Recall from Week 3 that functions introduce new scopes via a new local symbol table. An inner function is only in scope inside of the outer function, so this type of function definition is usually only used when the inner function is being returned to the outside world.

```Python
def outer():
    def inner(a):
        return a
    return inner

f = outer()
print(f)  # <function outer.<locals>.inner at 0x1044b61e0>
print(f(10))  # => 10

f2 = outer()
print(f2)  # <function outer.<locals>.inner at 0x1044b6268> (Different from above!)
print(f2(11))  # => 11
```

Why are the memory addresses different for `f` and `f2`? Discuss with a partner.

In [ ]:
def outer():
    def inner(a):
        return a
    return inner

f = outer()
print(f)  # <function outer.<locals>.inner at 0x1044b61e0>
print(f(10))  # => 10

f2 = outer()
print(f2)  # <function outer.<locals>.inner at 0x1044b6268> (Different from above!)
print(f2(11))  # => 11

### Closure
As we saw above, the definition of the inner function occurs during the execution of the outer function. This implies that a nested function has access to the environment in which it was defined. Therefore, it is possible to return an inner function that remembers contents of the outer function, even after the outer function has completed execution. This model is referred to as a closure.

```Python
def make_adder(n):
    def add_n(m):  # Captures the outer variable `n` in a closure
        return m + n
    return add_n

add1 = make_adder(1)
print(add1)  # <function make_adder.<locals>.add_n at 0x103edf8c8>
add1(4)  # => 5
add1(5)  # => 6

add2 = make_adder(2)
print(add2)  # <function make_adder.<locals>.add_n at 0x103ecbf28>
add2(4)  # => 6
add2(5)  # => 7
```

The information in a closure is available in the function's `__closure__` attribute. For example:

```Python
closure = add1.__closure__
cell0 = closure[0]
cell0.cell_contents  # => 1 (this is the n = 1 passed into make_adder)
``` 

As another example, consider the function:

```Python
def foo(a, b, c=-1, *d, e=-2, f=-3, **g):
    def wraps():
        print(a, c, e, g)
    return wraps
``` 

The `print` call induces a closure of `wraps` over `a`, `c`, `e`, `g` from the enclosing scope of `foo`. Or, you can imagine that wraps "knows" that it will need `a`, `c`, `e`, and `g` from the enclosing scope, so at the time `wraps` is defined, Python takes a "screenshot" of these variables from the enclosing scope and stores references to the underlying objects in the `__closure__` attribute of the `wraps` function.

```Python
w = foo(1, 2, 3, 4, 5, e=6, f=7, y=2, z=3)
list(map(lambda cell: cell.cell_contents, w.__closure__))
# => [1, 3, 6, {'y': 2, 'z': 3}]
```

What happens in the following situation? Why?
```Python
def outer(l):
    def inner(n):
        return l * n
    return inner
    
l = [1, 2, 3]
f = outer(l)
print(f(3))  # => ??

l.append(4)
print(f(3))  # => ??
```

In [ ]:
def outer(l):
    def inner(n):
        return l * n
    return inner
    
l = [1, 2, 3]
f = outer(l)
print(f(3))  # => ??

l.append(4)
print(f(3))  # => ??

## Functions
Alright, back to functions! This stuff is really fun too! Let's start with an optional problem that puts together all of the things we've learned about functions so far.

### *Optional: Putting it all together*
*If you feel confident that you understand how function calling works, you can skip this section. We suggest that you work through it if you'd like more practice, but the final decision is up to you.*

Often, however, we don't just see keyword arguments of variadic parameter lists in isolated situations. The following function definition, which incorporates positional parameters, keyword parameters, variadic positional parameters, keyword-only default parameters and variadic keyword parameters, is valid Python code. 

```Python
def all_together(x, y, z=1, *nums, indent=True, spaces=4, **options):
    print("x:", x)
    print("y:", y)
    print("z:", z)
    print("nums:", nums)
    print("indent:", indent)
    print("spaces:", spaces)
    print("options:", options)
```

For each of the following function calls, predict whether the call is valid or not. If it is valid, what will the output be? If it is invalid, what is the cause of the error?

```Python
all_together(2)
all_together(2, 5, 7, 8, indent=False)
all_together(2, 5, 7, 6, indent=None)
all_together()
all_together(indent=True, 3, 4, 5)
all_together(**{'indent': False}, scope='maximum')
all_together(dict(x=0, y=1), *range(10))
all_together(**dict(x=0, y=1), *range(10))
all_together(*range(10), **dict(x=0, y=1))
all_together([1, 2], {3:4})
all_together(8, 9, 10, *[2, 4, 6], x=7, spaces=0, **{'a':5, 'b':'x'})
all_together(8, 9, 10, *[2, 4, 6], spaces=0, **{'a':[4,5], 'b':'x'})
all_together(8, 9, *[2, 4, 6], *dict(z=1), spaces=0, **{'a':[4,5], 'b':'x'})
```

In [ ]:
# Before running me, predict which of these calls will be invalid and which will be valid!
# For valid calls, what is the output?
# For invalid calls, why is it invalid?
def all_together(x, y, z=1, *nums, indent=True, spaces=4, **options):
    print("x:", x)
    print("y:", y)
    print("z:", z)
    print("nums:", nums)
    print("indent:", indent)
    print("spaces:", spaces)
    print("options:", options)
    
# Uncomment the ones you want to run!
# all_together(2)
# all_together(2, 5, 7, 8, indent=False)
# all_together(2, 5, 7, 6, indent=None)
# all_together()
# all_together(indent=True, 3, 4, 5)
# all_together(**{'indent': False}, scope='maximum')
# all_together(dict(x=0, y=1), *range(10))
# all_together(**dict(x=0, y=1), *range(10))
# all_together(*range(10), **dict(x=0, y=1))
# all_together([1, 2], {3:4})
# all_together(8, 9, 10, *[2, 4, 6], x=7, spaces=0, **{'a':5, 'b':'x'})
# all_together(8, 9, 10, *[2, 4, 6], spaces=0, **{'a':[4,5], 'b':'x'})
# all_together(8, 9, *[2, 4, 6], *dict(z=1), spaces=0, **{'a':[4,5], 'b':'x'})

Write at least two more instances of function calls, not listed above, and predict their output. Are they valid or invalid? Check your hypothesis.

In [ ]:
# Write two more function calls.
# all_together(...)
# all_together(...)

### Default Mutable Arguments - A Dangerous Game

A function's default values are evaluated at the point of function definition in the defining scope. For example:

In [ ]:
x = 5

def square(num=x):
    return num * num

x = 6
print(square())   # => 25, not 36
print(square(x))  # => 36

**Warning: A function's default values are evaluated *only once*, when the function definition is encountered. This is important when the default value is a mutable object such as a list or dictionary**

Predict what the following code will do, then run it to test your hypothesis:

```Python
def append_twice(a, lst=[]):
    lst.append(a)
    lst.append(a)
    return lst
   
# Works well when the keyword is provided
print(append_twice(1, lst=[4]))  # => [4, 1, 1]
print(append_twice(11, lst=[2, 3, 5, 7]))  # => [2, 3, 5, 7, 11, 11]

# But what happens here?
print(append_twice(1))
print(append_twice(2))
print(append_twice(3))
```

In [ ]:
# Something fishy is going on here. Can you deduce what is happening?
def append_twice(a, lst=[]):
    lst.append(a)
    lst.append(a)
    return lst
   
# Works well when the keyword is provided
print(append_twice(1, lst=[4]))  # => [4, 1, 1]
print(append_twice(11, lst=[2, 3, 5, 7]))  # => [2, 3, 5, 7, 11, 11]

# But what happens here?
print(append_twice(1))
print(append_twice(2))
print(append_twice(3))

After you run the code, you should see the following printed to the screen:

```
[1, 1]
[1, 1, 2, 2]
[1, 1, 2, 2, 3, 3]
```
Discuss with a partner why this is happening.

If you don’t want the default value to be shared between subsequent calls, you can use a sentinel value as the default value (to signal that no keyword argument was explicitly provided by the caller). If so, your function may look something like:

```Python
def append_twice(a, lst=None):
    if lst is None:
        lst = []
    lst.append(a)
    lst.append(a)
    return lst
```

Discuss with a partner whether you think this solution feels better or worse.

In [ ]:
def append_twice(a, lst=None):
    if lst is None:
        lst = []
    lst.append(a)
    lst.append(a)
    return lst

## Investigating Function Objects

In Monday's class, we mentioned that functions are objects, and that they might have interesting attributes to explore. We'll poke around several of these attributes more in depth here.

Usually, this information isn't particularly useful for practitioners (you'll rarely want to hack around with the internals of functions), but even seeing that you *can* in Python is very cool.

In this section, there is no code to write. Instead, you will be reading and running code and observing the output. Nevertheless, we encourage you to play around with the code cells to experiment and explore on your own.

#### Default Values (`__defaults__` and `__kwdefaults__`)

As stated earlier, any default values (either normal default arguments or the keyword-only default arguments that follow a variadic positional argument parameter) are bound to the function object at the time of function definition. Consider our `all_together` function from earlier, and run the following code. Why might the `__defaults__` attribute be a tuple, but the `__kwdefaults__` attribute be a dictionary?

In [ ]:
def all_together(x, y, z=1, *nums, indent=True, spaces=4, **options): pass

all_together.__defaults__  # => (1, )
all_together.__kwdefaults__  # => {'indent':True, 'spaces':4}

#### Documentation (`__doc__`)

The first string literal in any function, if it comes before any expression, is bound to the function's `__doc__` attribute. 

In [ ]:
def my_function():
    """Summary line: do nothing, but document it.
        
    Description: No, really, it doesn't do anything.
    """
    pass

print(my_function.__doc__)
# Summary line: Do nothing, but document it.
#
#     Description: No, really, it doesn't do anything.

As stated in lecture, lots of tools use these documentation strings to great advantage. For example, the builtin `help` function displays information from docstrings, and many API-documentation-generation tools like [Sphynx](http://www.sphinx-doc.org/en/stable/) or [Epydoc](http://epydoc.sourceforge.net/) use information contained in the docstring to form smart references and hyperlinks on documentation websites.

Furthermore, the [doctest](https://docs.python.org/3/library/doctest.html) standard library module, in it's own words, "searches [the documentation string] for pieces of text that look like interactive Python sessions, and then executes those sessions to verify that they work exactly as shown." Cool!

#### Code Object (`__code__`)

In CPython, the reference implementation of Python used by many people (including us), functions are byte-compiled into executable Python code, or _bytecode_, when defined. This code object, which represents the bytecode and some administrative information, is bound to the `__code__` attribute, and has a ton of interesting properties, best illustrated by example. Code objects are immutable and contain no references to immutable objects.

```Python
def all_together(x, y, z=1, *nums, indent=True, spaces=4, **options):
    """A useless comment"""
    print(x + y * z)
    print(sum(nums))
    for k, v in options.items():
        if indent:
            print("{}\t{}".format(k, v))
        else:
            print("{}{}{}".format(k, " " * spaces, v))
            
code = all_together.__code__
```

| Attribute  | Sample Value | Explanation |
| --- | --- | --- |
| `code.co_argcount` | `3` | number of positional arguments (including arguments with default values) |
| `code.co_cellvars` | `()` | tuple containing the names of local variables that are referenced by nested functions |
| `code.co_code` | `b't\x00\x00...\x04S\x00'` | string representing the sequence of bytecode instructions |
| `code.co_consts` | `('A useless comment', '{}\t{}', '{}{}{}', ' ', None)` | tuple containing the literals used by the bytecode - our `None` is from the implicit `return None` at the end |
| `code.co_filename` | `filename` or `<stdin>` or `<ipython-input-#-xxx>` | file in which the function was defined |
| `code.co_firstlineno` | `1` | line of the file the first line of the function appears |
| `code.co_flags` | `79` | AND of compiler-specific binary flags whose internal meaning is (mostly) opaque to us |
| `code.co_freevars` | `()` | tuple containing the names of free variables |
| `code.co_kwonlyargcount` | `2` | number of keyword-only arguments |
| `code.co_lnotab` | `b'\x00\x02\x10\x01\x0c\x01\x12\x01\x04\x01\x12\x02'` | string encoding the mapping from bytecode offsets to line numbers |
| `code.co_name` | `"all_together"` | the function name  |
| `code.co_names` | `('print', 'sum', 'items', 'format')` | tuple containing the names used by the bytecode |
| `code.co_nlocals` | `9` | number of local variables used by the function (including arguments) |
| `code.co_stacksize` | `7` | required stack size (including local variables) |
| `code.co_varnames` | `('x', 'y', 'z', 'indent', 'spaces', 'nums', 'options', 'k', 'v')` | tuple containing the names of the local variables (starting with the argument names) |

More info on this, and on all types in Python, can be found at the [data model reference](https://docs.python.org/3/reference/datamodel.html#the-standard-type-hierarchy). For code objects, you have to scroll down to "Internal Types."

In [ ]:
def all_together(x, y, z=1, *nums, indent=True, spaces=4, **options):
    """A useless comment"""
    print(x + y * z)
    print(sum(nums))
    for k, v in options.items():
        if indent:
            print("{}\t{}".format(k, v))
        else:
            print("{}{}{}".format(k, " " * spaces, v))
            
code = all_together.__code__

print(code.co_argcount)
print(code.co_cellvars)
print(code.co_code)
print(code.co_consts)
print(code.co_filename)
print(code.co_firstlineno)
print(code.co_flags)
print(code.co_freevars)
print(code.co_kwonlyargcount)
print(code.co_lnotab)
print(code.co_name)
print(code.co_names)
print(code.co_nlocals)
print(code.co_stacksize)
print(code.co_varnames)

##### Security

As we briefly mentioned in class, this can lead to a pretty glaring security vulnerability. Namely, the code object on a given function can be hot-swapped for the code object of another (perhaps malicious function) at runtime!

In [ ]:
def nice(): print("You're awesome!")
def mean(): print("You're... not awesome. OOOOH")

# Overwrite the code object for nice
nice.__code__ = mean.__code__

print(nice())  # prints "You're... not awesome. OOOOH"

##### `dis` module

The `dis` module, for "disassemble," exports a `dis` function that allows us to disassemble Python byte code (at least, for Python distributions implemented in CPython for existing versions). The disassembled code isn't exactly normal assembly code, but rather is a specialized Python syntax

```Python
def gcd(a, b):
    while b:
        a, b = b, a % b
    return a
    
import dis
dis.dis(gcd)
"""
  2           0 SETUP_LOOP              27 (to 30)
        >>    3 LOAD_FAST                1 (b)
              6 POP_JUMP_IF_FALSE       29

  3           9 LOAD_FAST                1 (b)
             12 LOAD_FAST                0 (a)
             15 LOAD_FAST                1 (b)
             18 BINARY_MODULO
             19 ROT_TWO
             20 STORE_FAST               0 (a)
             23 STORE_FAST               1 (b)
             26 JUMP_ABSOLUTE            3
        >>   29 POP_BLOCK

  4     >>   30 LOAD_FAST                0 (a)
             33 RETURN_VALUE
"""
```

Details on the instructions themselves can be found [here](https://docs.python.org/3/library/dis.html#python-bytecode-instructions).
You can read more about the `dis` module [here](https://docs.python.org/3/library/dis.html).

In [ ]:
def gcd(a, b):
    while b:
        a, b = b, a % b
    return a
    
import dis
dis.dis(gcd)

#### Parameter Annotations (`__annotations__`)

Python allows us to add type annotations on functions arguments and return values. This leads to a world of complex possibilities and is still fairly controversial in the Python ecosystem. Nevertheless, it can be used to communicate to your clients expectations for the types of arguments.

Importantly, Python doesn't actually do anything with these annotations and will not check that supplied arguments conform to the type hint specified. This language feature is only made available through the collection of function annotations.

In [ ]:
def annotated(a: int, b: str) -> list:
    return [a, b]

print(annotated.__annotations__)
# => {'b': <class 'str'>, 'a': <class 'int'>, 'return': <class 'list'>}

This information can be used to build some really neat runtime type-checkers for Python!

For more info, check out [PEP 3107](https://www.python.org/dev/peps/pep-3107/) on function annotations or [PEP 484](https://www.python.org/dev/peps/pep-0484/) on type hinting (which was introduced in Python 3.5)

#### Call (`__call__`)

All Python functions have a `__call__` attribute, which is the actual object called when you use parentheses to "call" a function. That is,

In [ ]:
def greet(): print("Hello world!")

greet() # "Hello world!"
# is just syntactic sugar for
greet.__call__()  # "Hello world!"

This means that any object (including instances of custom classes) with a `__call__` method can use the parenthesized function call syntax! For example, we can construct a callable `Polynomial` class. We haven't talked about class syntax yet, so feel free to skip this example.

```Python
class Polynomial:
    def __init__(self, coeffs):
        """Store the coefficients..."""
        
    def __call__(self, x):
        """Compute f(x)..."""


# The polynomial f(x) = 4 + 4 * x + 4 * x ** 2
f = Polynomial(4, 4, 1)
f(5)  # Really, this is f.__call__(5)
```

We'll see a lot more about using these so-called "magic methods" to exploit Python's apparent operators (like function calling, `+` (`__add__`) or `*` (`__mul__`), etc) in Week 5.

#### Name Information (`__module__`, `__name__`, and `__qualname__`)

Python functions also store some name information about a function, generally for the purposes of friendly printing.

`__module__` refers to the module that was active at the time the function was defined. Any functions defined in the interactive interpreter, or run as as a script, will have `__module__ == '__main__'`, but imported modules will have their `__module__` attribute set to the module name. For example, `math.sqrt.__module__` is `"math"`.

`__name__` is the function's name. Nothing special here.

`__qualname__`, which stands for "qualified name," only differs from `__name__` when you're dealing with nested functions, which we'll talk about more Week 4.


#### Closure (`__closure__`)

If you're familiar with closures in other languages, Python closures work almost the exact same way. Closures really only arise when dealing with nested functions, so we'll see more Week 4. This bit of text is just to give you a teaser for what's coming soon - yes, Python has closures!

#### `inspect` module

As a brief note, all of this mucking around with the internals of Python functions can't be good for our health. Luckily, there's a standard library module for this! The `inspect` module gives us a lot of nice tools for interacting not only with the internals of functions, but also the internals of a lot of other types as well. Check out [the documentation](https://docs.python.org/3/library/inspect.html) for some nice examples.

In [ ]:
import inspect

def all_together(x, y, z=1, *nums, indent=True, spaces=4, **options): pass

print(inspect.getfullargspec(all_together))

## Finished Early?
Wow! Uh... this is all we've got for you. So at this point, feel free to call a TA over, have them sign off on your work, and then you're free to go!

If you'd still like to stay in lab, though, and you didn't get a chnace to read through the following documents last week, though, now is a perfectly good time to peruse them: scan through [PEP 8](https://www.python.org/dev/peps/pep-0008/), Python's official style guide, as well as [PEP 257](https://www.python.org/dev/peps/pep-0257/), Python's suggestions for docstring conventions, if you didn't get a chance to read them last week.

> With &#129412;s by @psarin and @coopermj